# Entanglement forging

In this tutorial, we show how to simulate entanglement forging for a water molecule at equilibrium.

In [1]:
import pyscf
import pyscf.mcscf
import ffsim
import math

# Build a water molecule
radius_1 = 0.958  # position for the first H atom
radius_2 = 0.958  # position for the second H atom
bond_angle_deg = 104.478  # bond angles.

h1_x = radius_1
h2_x = radius_2 * math.cos(math.pi / 180 * bond_angle_deg)
h2_y = radius_2 * math.sin(math.pi / 180 * bond_angle_deg)

mol = pyscf.gto.Mole()
mol.build(
    atom=[
        ["O", (0, 0, 0)],
        ["H", (h1_x, 0, 0)],
        ["H", (h2_x, h2_y, 0)],
    ],
    basis="sto-6g",
    symmetry="c2v",
)

# Define active space
active_space = range(1, mol.nao_nr())

# Get molecular data and molecular Hamiltonian (one- and two-body tensors)
mol_data = ffsim.MolecularData.from_mole(mol, active_space=active_space, fci=True)
norb = mol_data.norb
nelec = mol_data.nelec
mol_hamiltonian = mol_data.hamiltonian

converged SCF energy = -75.6787887956297
CASCI E = -75.7288249991515  E(CI) = -23.6332495815006  S^2 = 0.0000000


In [2]:
import numpy as np


# Construct ansatz operator
def brickwork(norb: int, n_layers: int):
    for i in range(n_layers):
        for j in range(i % 2, norb - 1, 2):
            yield (j, j + 1)


n_layers = norb
interaction_pairs = list(brickwork(norb, n_layers))
rng = np.random.default_rng(1234)
thetas = rng.uniform(-np.pi, np.pi, size=len(interaction_pairs))
operator = ffsim.HopGateAnsatzOperator(norb, interaction_pairs, thetas)

# Compute energy of ansatz state
reference_occupations_spatial = [(0, 1, 2, 3), (1, 2, 3, 4), (0, 1, 2, 4)]
reference_occupations = list(
    zip(reference_occupations_spatial, reference_occupations_spatial)
)
energy, ansatz_state = ffsim.multireference_state(
    mol_hamiltonian, operator, reference_occupations, norb=norb, nelec=nelec
)

print(f"Energy at initialialization: {energy}")

Energy at initialialization: -74.20656273321596


In [3]:
import scipy.optimize


def fun(x):
    # Initialize the ansatz operator from the parameter vector
    operator = ffsim.HopGateAnsatzOperator(norb, interaction_pairs, x)
    # Compute energy
    energy, _ = ffsim.multireference_state(
        mol_hamiltonian, operator, reference_occupations, norb=norb, nelec=nelec
    )
    return energy


result = scipy.optimize.minimize(
    fun, x0=operator.thetas, method="L-BFGS-B", options=dict(maxfun=100)
)

print(f"Number of parameters: {len(result.x)}")
print(result)

Number of parameters: 15
  message: STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
  success: False
   status: 1
      fun: -75.68085225170408
        x: [ 2.996e+00 -7.549e-01 ...  2.650e+00  8.012e-01]
      nit: 6
      jac: [ 1.756e-03  9.118e-03 ... -1.192e-02  9.521e-04]
     nfev: 112
     njev: 7
 hess_inv: <15x15 LbfgsInvHessProduct with dtype=float64>
